# Exp 4: Exploring SMILLE
This dataset anntotates EFCAMDAT data with grammar constructions. This experiment aims to understand the structure of the data and verify the match of the IDs.

In [1]:
import pandas as pd
DATA_PATH = "../data/"

Load SMILLE data (annotation of grammar in EF-CAMDAT)

In [2]:
smille = pd.read_csv(f"{DATA_PATH}cambridge.smille.filtered.csv")

Check the available columns

In [5]:
for col in smille.columns:
    print(col)

id
ped_Short_Answers_C1
syn_WC_ATI
ped_VBT_PTS_(review)_The_Past_Tense_(review)
ped_Demonstratives_A1
ped_Question_Tags_B1
ped_The_Present_Tense_A2
syn_VBT_CONT
ped_The_Present_Tense_A1
ped_MDV_MVI_Modals
ped_WC_ATI_(review)_Infinitive_and__ing_Forms_(review)
ped_QA_WHQ_Interrogatives
syn_WC_PPAR
ped_LEX_NNP_Nouns
ped_QA_SASA_Short_Answers
syn_WC_RINF
ped_Infinitive_and_-ing_Forms_(advanced)_C1
ped_WC_GEX_Infinitive_and__ing_Forms
mor_content words
syn_SCF_VB
ped_Imperative_Sentences_A2
syn_CONN_TIM
ped_LEX_REF_Personal_and_Reflexive_Pronouns
mor_LEX_DEM
ped_LEX_NUM_Numerals
ped_QA_QTAG_Short_Answers_and_Responses
ped_QA_SADD_The_Past_Tense
ped_QA_QTAG_(review)_Question_Tags_(review)
ped_LEX_QUB_Quantifiers
ped_VBT_FTP_The_Future_Forms
ped_Numerals_A1
ped_LEX_SHF_Short_Forms
ped_QA_SAH_The_Present_Tense
ped_WC_LINF_Infinitive_and__ing_Forms
ped_VBT_PRS_(review)_The_Present_Tense_(review)
ped_Adjectives_A1
ped_Prepositional_and_Phrasal_Verbs_B2
ped_VBT_PRP_(review)_The_Present_Perfect_T

## Read EF-CAMDAT

In [ ]:
efcamdat_path = f"{DATA_PATH}EFCAMDAT_Database.xml"

Sanitize the EF-CAMDAT to be able to read it in without errors

In [15]:
# Fix an error due to the <br> tag
# Open the original file in read mode and a new file in write mode
with open(efcamdat_path, 'r', encoding='utf-8') as file:
    file_content = file.read()

# Replace all occurrences of <br> with <br />
updated_content = file_content.replace('<br>', '<br />')

# Write the updated content back to the file or a new file
with open(efcamdat_path, 'w', encoding='utf-8') as file:
    file.write(updated_content)

# Line numbers with errorneous tags
lines_to_remove = list(range(5080477, 5080486))


with open(efcamdat_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

for line in lines[lines_to_remove]:
    print(line)
# CAUTION: do not execute this twice, it will remove other lines then
#del lines[lines_to_remove] # remove them

# Write the modified content back to the file
with open(efcamdat_path, 'w', encoding='utf-8') as file:
    file.writelines(lines)


    <writing id="576978" level="4" unit="2">

      <learner id="165703" nationality="it"/>

      <topic id="26">Describing routines</topic>

      <date>2013-01-19 05:24:16.490</date>

      <grade>90</grade>

    <text>

      Dear <change><selection>MRS</selection><tag><symbol>SP</symbol><correct>Mrs</correct></tag></change> <change><selection>Deebie</selection><tag><symbol>SP</symbol><correct>Debbie</correct></tag></change>,<change><selection></selection><tag><symbol>AS</symbol><correct> </correct></tag></change></code>thank you for your help<change><selection>.<change><selection></selection><tag><symbol>AS</symbol><correct> </correct></tag></change></selection><tag><symbol>PU</symbol><correct>.</correct></tag></change>This is my family's routine: My mother gets up at 7 am<change><selection>.<change><selection></selection><tag><symbol>AS</symbol><correct> </correct></tag></change></selection><tag><symbol>PU</symbol><correct>.</correct></tag></change>She mops the floor<change><sele

Load EF-CAMDAT to try and match with the SMILLE output

In [16]:
efcamdat = pd.read_xml(efcamdat_path, xpath="//writing")

Let's print the IDs

In [18]:
efcamdat.id

0                1
1                2
2                3
3                4
4                5
            ...   
1180303    1211598
1180304    1211599
1180305    1211600
1180306    1211601
1180307    1211602
Name: id, Length: 1180308, dtype: int64

Unfortunately, it looks like these IDs do not match with this SMILLE data.